In [26]:
import pickle
from pathlib import Path
import httpx
import random
import json
import numpy as np
import asyncio
from multiprocessing import Pool

In [ ]:
import sys
#!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install --user matplotlib
#!{sys.executable} -V

In [ ]:
#vector_path=pickle.load('C:\PRojetos\STF-Digital\IA\agrupament\data\vetores')

In [2]:
%%time
p=Path('..','..','agrupamento','data','vetores')
vector = None
for f in p.glob('*'):
    vector = pickle.load(f.open(mode='rb'))

CPU times: total: 93.8 ms
Wall time: 142 ms


<timed exec>:4: ResourceWarning: unclosed file <_io.BufferedReader name='..\\..\\agrupamento\\data\\vetores\\vetores_1000_limpeza_basica_clean.pkl'>


In [3]:
#glob
v_qtd = len(vector)
print('Quantidade de vetores', v_qtd)
v_len = len(vector[0])
print('Tamanho do vetor', v_len)
query_vector = vector[10]

Quantidade de vetores 22868
Tamanho do vetor 300


# Definição do Indice

Definição do formato do índice e credenciais de acesso ao elastic

In [4]:
index = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
  "mappings": {
    "properties": {
      "my_vector": {
        "type": "dense_vector",
        "dims": v_len,
        "index": "true",
        "similarity": "dot_product"
      },
      "seq_documento" : {
        "type" : "keyword"
      }
    }
  }
}

username='elastic'
password='password'
headers = {'Content-Type': 'application/json'}

index = json.dumps(index)

## Inicialização do elastic localmente

O Elastic recomenda cria uma rede antes

```docker network create pocnet```

Para iniciar o container

```docker run -d --name elasticsearch --net pocnet -p 9200:9200 -p 9300:9300 -e "ELASTIC_PASSWORD=password" -e "discovery.type=single-node" elasticsearch:8.4.2```

Iniciei também o ElasticHQ para acompannhar as estatísticas do ambiente com o comando

```docker run -d --name elastichq --net pocnet --rm -p 5000:5000 -e HQ_VERIFY_CERTS=False -e HQ_ENABLE_SSL=False -e HQ_DEBUG=True elastichq/elasticsearch-hq```


## Criação do íncide poc_index

In [5]:
%%timeit -r 1 -n 1

req = requests.request(
    'PUT', 
    'https://localhost:9200/poc_index', 
    auth=(username, password), 
    data=index,
    verify=False, 
    headers=headers
)
print(req.status_code)
print(req.headers)
#print(req.content)

C:\Users\aurelio.costa\AppData\Roaming\Python\Python39\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


200
{'X-elastic-product': 'Elasticsearch', 'content-type': 'application/json', 'content-length': '68'}
528 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


# Testando a indexação de documentos

In [6]:
%%time

for idx, doc in enumerate(vector):
    unit_vector = doc / np.linalg.norm(doc)
    #unit_vector = unit_vector + np.abs(min(unit_vector))
    resp = requests.request(
        'POST', 
        'https://localhost:9200/poc_index/_doc',
        auth=(username, password), 
        data=json.dumps({'my_vector': unit_vector.tolist(), 'seq_documento': idx}),
        verify=False, 
        headers=headers
    )
    #print(idx,resp.status_code, end=' | ')
    #print(resp.headers)
    #print(resp.content)
print('')


CPU times: total: 8min 35s
Wall time: 17min 22s


In [19]:
v0 = random.sample(vector, 1)[0]
v=v0/np.linalg.norm(v0)
query_vector = v #(v+ np.abs(min(v))).tolist()
print('Norma de v0', np.linalg.norm(v0))
print('Norma de v (deve ser 1)', np.linalg.norm(v))
print('Norma de query_vector ', np.linalg.norm(query_vector))
for pair in zip(v0, v):
    print(f'{pair[0]:>20}|{pair[1]:<20}')

Norma de v0 38.55699
Norma de v (deve ser 1) 1.0
Norma de query_vector  1.0
 -1.6490223407745361|-0.042768437415361404
   4.013855934143066|0.10410189628601074 
  -2.315446376800537|-0.06005256995558739
  -2.494363784790039|-0.0646929070353508 
   2.394704818725586|0.06210818514227867 
  1.9350900650024414|0.05018778517842293 
 -1.5341722965240479|-0.039789728820323944
 -0.6363174915313721|-0.01650329679250717
  0.9395543336868286|0.024367935955524445
  0.1273333728313446|0.003302471712231636
-0.21944911777973175|-0.005691552069038153
  2.5611140727996826|0.06642411649227142 
 -0.9911512136459351|-0.025706134736537933
   2.693150281906128|0.06984855979681015 
 -1.6365984678268433|-0.04244621843099594
 0.48573046922683716|0.012597727589309216
 -0.4805244505405426|-0.012462706305086613
   0.950190544128418|0.02464379370212555 
  1.6916686296463013|0.04387449845671654 
   2.097315788269043|0.05439521372318268 
  -3.974111795425415|-0.10307110846042633
  1.8993240594863892|0.04926017299294

# Deletando o indice

In [ ]:
%%timeit -r 1 -n 1
with httpx.Client(verify=False) as client:
    resp = client.request(
        'DELETE', 
        'https://localhost:9200/poc_index',
        auth=(username, password), 
        headers=headers
    )
print(resp.status_code)
print(resp.headers)
#print(resp.content)

# Teste de consultas

In [29]:
# Consulta o campo my_vector usando dotProduct com o parametro query_vector
query = {
    "query": {
      "script_score": {
        "query": {"match_all": {}},
        "script": {
          "source": "dotProduct(params.query_vector, 'my_vector') + 10.0",
          "params": {"query_vector": query_vector.tolist()}
        }
      }
    }
}

# Transforma o dicionario da consulta em um json para enviar via HTTP
query = json.dumps(query)

with httpx.Client(verify=False) as client:
    resp = client.request(
        'GET', 
        'https://localhost:9200/poc_index/_search',
        data=query,
        auth=(username, password), 
        headers=headers
    )
print(resp.content)

#content = json.loads(resp.content)

b'{"took":1179,"timed_out":false,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0},"hits":{"total":{"value":10000,"relation":"gte"},"max_score":11.0,"hits":[{"_index":"poc_index","_id":"WAM7poQBS4w7sqjB5zpH","_score":11.0,"_source":{"my_vector": [-0.042768437415361404, 0.10410189628601074, -0.06005256995558739, -0.0646929070353508, 0.06210818514227867, 0.05018778517842293, -0.039789728820323944, -0.01650329679250717, 0.024367935955524445, 0.003302471712231636, -0.005691552069038153, 0.06642411649227142, -0.025706134736537933, 0.06984855979681015, -0.04244621843099594, 0.012597727589309216, -0.012462706305086613, 0.02464379370212555, 0.04387449845671654, 0.05439521372318268, -0.10307110846042633, 0.04926017299294472, 0.09363048523664474, -0.08645632863044739, -0.014706489630043507, 0.023004140704870224, 0.006219847593456507, 0.045580584555864334, -0.025700753554701805, -0.007671655155718327, -0.08825919777154922, 0.021147415041923523, -0.11143796145915985, -0.09582529217004776

## Busca usando o a API kNN

In [35]:

def retorna_similares(vector, k, threshold=0.9, candidates=200):
    knn_query = {
        "knn": {
            "field": "my_vector",
            "query_vector": vector,
            "k": k,
            "num_candidates": candidates
        },
        "size": k,
        "fields": ['seq_documento']
    }

    with httpx.Client(verify=False, timeout=10) as client:
        resp = client.request(
            'GET', 
            'https://localhost:9200/poc_index/_search',
            data=json.dumps(knn_query),
            auth=(username, password), 
            headers=headers
        )

    content = json.loads(resp.content)
    ret = set()
    try:
        time_taken=content['took']
        hits_count=len(content['hits']['hits'])

        #print(f'Levou {time_taken}ms o melhor score de {hits_count}')
        
        for hit in content['hits']['hits']:
            score=hit["_score"]
            seq_documento=hit["fields"]["seq_documento"][0]
            if float(score) < threshold:
                break
            #print(f'{seq_documento} - score {score}')
            ret.add(seq_documento)
    except KeyError as err:
        print("KeyError")
        print(content)
    return ret



In [28]:
%%time

similares = retorna_similares(query_vector.tolist(), k=50)
print(similares)

{'6441'}
CPU times: total: 93.8 ms
Wall time: 289 ms


## Clusterizando os documentos

Listamos os objetos no índice e depois percorremos essa lista para criar o cluster para cada um dos vetores

In [21]:
# Pega a quantidade de documentos no indice
with httpx.Client(verify=False) as client:
    resp = client.request(
        'GET', 
        'https://localhost:9200/poc_index/_count',
        auth=(username, password), 
        headers=headers
    )     
content = json.loads(resp.content)
num_docs = content['count']
print('Numero de documentos no índice', num_docs)

22868


In [34]:

def cria_clusters(doc):
    query_seq_documento = {
       "_source": "my_vector"
    }
    
    with httpx.Client(verify=False, timeout=10) as client:
        resp = client.request(
            'GET', 
            'https://localhost:9200/poc_index/_search/?q=seq_documento:'+str(doc),
            auth=(username, password), 
            data=json.dumps(query_seq_documento),
#            verify=False, 
            headers=headers
        )     
    content = json.loads(resp.content)
    vec=content['hits']['hits'][0]['_source']['my_vector']
    
    similares = retorna_similares(vec, k=50)
    return similares




In [39]:
%%time

clusters=dict() # dicionario no formato cluster[seq_documento]= [conjunto de similares]

for doc in range(num_docs):
    try:
        clusters[doc] = cria_clusters(doc)
    except httpx.ConnectTimeout as err:
        print('Erro de time out ao processar documento', str(doc))


Erro de time out ao processar documento 5829
CPU times: total: 7min 37s
Wall time: 38min 51s


In [ ]:
%%time
print(clusters[100])
for k1,v1 in clusters.items():
    for k2,v2 in clusters.items():
        if v1 == v2 and k1 != k2:
            print("Similaries", k1, k2)

In [ ]:
%matplotlib inline
#from collections import defaultdict
#hist = defaultdict(lambda x: 0)
#hist = dict()
hist = list()
for k,v in clusters.items():
    hist.append(len(await v))

hist.sort(reverse=True)

import matplotlib.pyplot as plt

In [ ]:
n, bins, patches = plt.hist(hist, 50)
#plt.show()

In [ ]:
hist[:2]